In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download("stopwords")
nltk.download("wordnet")
import re
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from genism.models import Word2vec

import spacy


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
"""
df = pd.read_csv('IMDB Dataset.csv')
lem = WordNetLemmatizer()
stem = PorterStemmer()
def rm_special_symbols_numbers(text):
    return re.sub(r'[^a-zA-Z\s]',"",text)
def pre_process(text):


    token = nltk.word_tokenize(text.lower())

    token = [lem.lemmatize(word) for word in token if word not in stopwords.words("english")]

    return " ".join(token)
pre = lambda text: [lem.lemmatize(word) for word in tqdm(nltk.word_tokenize(text.lower())) if word not in stopwords.words("english")]
df["review"] = df["review"].apply(rm_special_symbols_numbers)
df["review"] = df["review"].apply(pre)"""

'\ndf = pd.read_csv(\'IMDB Dataset.csv\')\nlem = WordNetLemmatizer()\nstem = PorterStemmer()\ndef rm_special_symbols_numbers(text):\n    return re.sub(r\'[^a-zA-Z\\s]\',"",text)\ndef pre_process(text):\n\n\n    token = nltk.word_tokenize(text.lower())\n\n    token = [lem.lemmatize(word) for word in token if word not in stopwords.words("english")]\n\n    return " ".join(token)\npre = lambda text: [lem.lemmatize(word) for word in tqdm(nltk.word_tokenize(text.lower())) if word not in stopwords.words("english")]\ndf["review"] = df["review"].apply(rm_special_symbols_numbers)\ndf["review"] = df["review"].apply(pre)'

In [ ]:
df = pd.read_csv("cleaned_text.csv")

In [ ]:
df

,Unnamed: 0,review,sentiment
0,0,"['one', 'reviewer', 'mentioned', 'watching', '...",positive
1,1,"['wonderful', 'little', 'production', 'br', 'b...",positive
2,2,"['thought', 'wonderful', 'way', 'spend', 'time...",positive
3,3,"['basically', 'there', 'family', 'little', 'bo...",negative
4,4,"['petter', 'matteis', 'love', 'time', 'money',...",positive
...,...,...,...
49995,49995,"['thought', 'movie', 'right', 'good', 'job', '...",positive
49996,49996,"['bad', 'plot', 'bad', 'dialogue', 'bad', 'act...",negative
49997,49997,"['catholic', 'taught', 'parochial', 'elementar...",negative
49998,49998,"['im', 'going', 'disagree', 'previous', 'comme...",negative


In [ ]:
df_p = df[df["sentiment"]=="positive"].iloc[0:2000]

df_n = df[df["sentiment"] == "negative"].iloc[0:2000]

df = pd.concat([df_p,df_n],axis=0)

In [ ]:
import ast


df["review"] = df["review"].apply(lambda x: ast.literal_eval(x))
df["review"]= df["review"].apply(' '.join)

In [ ]:
df

,Unnamed: 0,review,sentiment
0,0,one reviewer mentioned watching oz episode you...,positive
1,1,wonderful little production br br filming tech...,positive
2,2,thought wonderful way spend time hot summer we...,positive
4,4,petter matteis love time money visually stunni...,positive
5,5,probably alltime favorite movie story selfless...,positive
...,...,...,...
3953,3953,let see advantage watching piranha piranha wel...,negative
3954,3954,one thing produced movie feel later movie inte...,negative
3956,3956,contains spoiler movie plot summarized sentenc...,negative
3959,3959,high school track field athelete laura remstea...,negative


In [ ]:
X = df["review"]
y = df["sentiment"]

# DL with TFIDF

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df["review"])
y=df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# ML with Random Forest Classifier

In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier

# Train model
model = RandomForestClassifier()
model.fit(X, y)

# Save the model
with open("model.pkl", "wb") as file:
    pickle.dump(model, file)
print("Model saved as model.pkl")

Model saved as model.pkl


In [ ]:
# Run in bash
# python train_model.py

In [ ]:
# Ensure that X_train and X_test are in dense format
X_train_dense = X_train.toarray()  # Convert sparse matrix to dense matrix
X_test_dense = X_test.toarray()

# Reshape the data to match the RNN input shape (batch_size, timesteps, features)
X_train = np.expand_dims(X_train_dense, axis=-1)
X_test = np.expand_dims(X_test_dense, axis=-1)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train).astype('float32')
y_test = le.transform(y_test).astype('float32')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, SimpleRNN, Dropout, Reshape, Dense
from sklearn.metrics import classification_report

In [ ]:
# Build RNN

rnn_model = Sequential([
    Reshape((X_train.shape[1],1), input_shape=(X_train.shape[1],)),
    SimpleRNN(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Build LSTM

lstm_model = Sequential([
    Reshape((X_train.shape[1],1), input_shape=(X_train.shape[1],)),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

In [ ]:
# Build bILSTM

bi_lstm_model = Sequential([
    Reshape((X_train.shape[1],1), input_shape=(X_train.shape[1],)),
    Bidirectional(LSTM(128, return_sequences=False)),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

In [ ]:
model = rnn_model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(X_train,y_train, batch_size=32, validation_data=(X_test,y_test), verbose=1)#epochs=5

100/100 ━━━━━━━━━━━━━━━━━━━━ 652s 7s/step - accuracy: 0.4980 - loss: 0.7719 - val_accuracy: 0.5275 - val_loss: 0.6916


In [21]:
pred  = model.predict(X_test)
pred = (pred > 0.5).astype(int)

25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step


In [22]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       378
         1.0       0.53      1.00      0.69       422

    accuracy                           0.53       800
   macro avg       0.26      0.50      0.35       800
weighted avg       0.28      0.53      0.36       800



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model = lstm_model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(X_train,y_train, batch_size=32, validation_data=(X_test,y_test), verbose=1)#epochs=5


In [ ]:
pred  = model.predict(X_test)
pred = (pred > 0.5).astype(int)
print(classification_report(y_test, pred))

In [ ]:
model = bi_lstm_model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(X_train,y_train, batch_size=32, validation_data=(X_test,y_test), verbose=1)#epochs=5
pred  = model.predict(X_test)
pred = (pred > 0.5).astype(int)
print(classification_report(y_test, pred))

# DL with Word Vectors

In [ ]:
df_p = df[df["sentiment"]=="positive"].iloc[0:2500]

df_n = df[df["sentiment"] == "negative"].iloc[0:2500]

df_filter = pd.concat([df_p,df_n],axis=1)

In [ ]:
pip install fasttext

In [ ]:
import fasttext.util
import numpy as np

# Load pre-trained FastText model
# Load pre-trained FastText English word vectors
fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')

def generate_ngrams(words, n):
    """Generate word n-grams from a list of words."""
    return [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]

def sentence_embedding(sentence, n=1):
    """Compute the sentence embedding using FastText word or n-gram vectors."""
    words = sentence.split()
    ngrams = words if n == 1 else generate_ngrams(words, n)

    embeddings = [ft.get_word_vector(ngram) for ngram in ngrams if ngram in ft.words]

    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(300)  # Return zero vector if no valid words found

# Example usage:
sentence = "The quick brown fox jumps over the lazy dog"
embedding = sentence_embedding(sentence, n=2)  # Using bigrams
print(embedding.shape)  # Should print (300,)


In [ ]:
X = np.array([sentence_embedding(text) for text in df_filter["review"]])
y = df_filter["sentiment"]


# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Ensure that X_train and X_test are in dense format
X_train_dense = X_train.toarray()  # Convert sparse matrix to dense matrix
X_test_dense = X_test.toarray()

# Reshape the data to match the RNN input shape (batch_size, timesteps, features)
X_train = np.expand_dims(X_train_dense, axis=-1)
X_test = np.expand_dims(X_test_dense, axis=-1)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train).astype('float32')
y_test = le.transform(y_test).astype('float32')

In [ ]:
# Build RNN

rnn_model = Sequential([
    Reshape((X_train.shape[1],1), input_shape=(X_train.shape[1],)),
    SimpleRNN(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

# Build LSTM

lstm_model = Sequential([
    Reshape((X_train.shape[1],1), input_shape=(X_train.shape[1],)),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

# Build bILSTM

bi_lstm_model = Sequential([
    Reshape((X_train.shape[1],1), input_shape=(X_train.shape[1],)),
    Bidirectional(LSTM(128, return_sequences=False)),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])


In [ ]:
model = rnn_model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(X_train,y_train, batch_size=32, validation_data=(X_test,y_test), verbose=1)#epochs=5

pred  = model.predict(X_test)
pred = (pred > 0.5).astype(int)
print(classification_report(y_test, pred))

In [ ]:
model = lstm_model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(X_train,y_train, batch_size=32, validation_data=(X_test,y_test), verbose=1)#epochs=5

pred  = model.predict(X_test)
pred = (pred > 0.5).astype(int)
print(classification_report(y_test, pred))

In [ ]:

model = bi_lstm_model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(X_train,y_train, batch_size=32, validation_data=(X_test,y_test), verbose=1)#epochs=5
pred  = model.predict(X_test)
pred = (pred > 0.5).astype(int)
print(classification_report(y_test, pred))

# Using AI Model - BERT

In [ ]:
from transformers import AutoTokenizer

# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Rename columns for compatibility
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")


In [ ]:
from transformers import TrainingArguments, Trainer

# Define training parametersgmail
training_args = TrainingArguments(
    output_dir="./bert-custom",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,from tra
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    prediction = outputs.logits.argmax().item()
    return "Positive" if prediction == 1 else "Negative"

# Example reviews
reviews = [
    "Amazing movie with a great storyline!",
    "I didn't enjoy this film at all.",
    "It was okay, nothing special."
]

for review in reviews:
    print(f"Review: {review}\nPredicted Sentiment: {predict_sentiment(review)}\n")


In [ ]:
import pickle

# Save model and vectorizer
with open("random_forest_sentiment.pkl", "wb") as f:
    pickle.dump((model, tfidf), f)

print("Model saved successfully!")
